In [15]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.chat_models import ChatCohere

In [16]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
import langchain
langchain.verbose=True

In [17]:
from dotenv import load_dotenv
import os

import dotenv
dotenv.load_dotenv()

True

In [18]:
COHERE_API_KEY = os.environ["COHERE_API_KEY"]

In [19]:
cohere = ChatCohere(model="command", temperature = 0, streaming=True, verbose=True)

In [20]:
from langchain.retrievers.tavily_search_api import TavilySearchAPIRetriever
from langchain.retrievers import CohereRagRetriever
from langchain_community.chat_models import ChatCohere
from langchain_core.documents import Document

In [21]:
from tools import *

In [22]:
tools = [buy_stock, sell_stock, mean_reversion, rag]

In [23]:
from langchain.agents import create_structured_chat_agent

In [24]:
prompt = hub.pull("kenwu/react-json")

In [25]:
# Construct the JSON agent
agent = create_structured_chat_agent(cohere, tools, prompt)

In [26]:
agent_executor.invoke({"input": "Buy 10 shares of MSFT\n"})



> Entering new AgentExecutor chain...
Thought: The ticker symbol for Microsoft is "MSFT". We can buy 10 shares of MSFT using the buy_stock tool. 

Action:
```
{
  "action": "buy_stock",
  "action_input": {
    "ticker": "MSFT",
    "amount": 10
  }
}
```
Observation: 10 shares of MSFT is BOUGHT. ACTION COMPLETED
Thought: The buy_stock tool was used to purchase 10 shares of MSFT. The output should reflect that the purchase has been completed. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "10 shares of MSFT have been purchased."
}
```

> Finished chain.


{'input': 'Buy 10 shares of MSFT\n',
 'output': '10 shares of MSFT have been purchased.'}